In [3]:
# Load sentence retrieval result
model_name = ["BAAI/bge-large-en-v1.5", "intfloat/multilingual-e5-large", "facebook/contriever", "BAAI/bge-m3-unsupervised"][0]
model_raw_name = model_name.split('/')[-1]
dataset_name = ["nfcorpus", "trec-covid"][0]
dropout=["", "-0.02", "-0.05"][1]
temp = [0.01, 0.05, 0.1][1]
txt_name = f"{model_raw_name}/{dataset_name}-result{dropout}-temp-{temp}" # "bge_unsup_ood_dropout_0.02_temp_0.02.txt" # "bge_unsup_ood_dropout_0.02_temp_0.02.txt"
print(txt_name)

bge-large-en-v1.5/nfcorpus-result-0.02-temp-0.05


In [ ]:
# Load retrieval result
import json
q2d_retrieval = f"/gallery_louvre/jinyoung.kim/ood/retrieval/results/{model_raw_name}/{dataset_name}-q2d-top-300.jsonl"
with open(q2d_retrieval) as f:
    file = [json.loads(i) for i in f.readlines()]

In [27]:
import pandas as pd
from collections import defaultdict

retrieval_results = {i["_id"]: [i["_id"] for i in i["retrieval"][:100]] for i in file}

# Load queries that don't match failure documents
tsv_path = f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/qrels/test.tsv"
df = pd.read_csv(tsv_path, sep='\t')
df = df[df["score"] >= 1]
query_ids = df['query-id'].tolist()
corpus_ids = df['corpus-id'].tolist()
scores = df['score'].tolist()

query_dict = defaultdict(list)
for query_id, corpus_id, score in zip(query_ids, corpus_ids, scores):
    query_dict[query_id] = corpus_id
related_doc_ids = set(corpus_ids)

# Load recall result
path = f"/gallery_louvre/dayoon.ko/research/sds/eval_retrieval/retrieval/results/{model_raw_name}/{dataset_name}-n-query-mt-2.csv"
df = pd.read_csv(path)
df = df.drop('Unnamed: 0', axis=1)
df = df.set_index("corpus-id")

In [55]:
# Wrong documents
wrong_doc_ids = []
for ret in retrieval_results.values():
    wrong_doc_ids.append(list(set(ret) - related_doc_ids))
wrong_doc_ids = sum(wrong_doc_ids, [])

wrong_doc_counts = defaultdict(int)
for i in wrong_doc_ids:
    wrong_doc_counts[i] += 1
sorted_wrong_doc_ids = sorted(set(wrong_doc_ids), key=lambda x: wrong_doc_counts[x], reverse=True)
sorted_wrong_doc_ids

['MED-939',
 'MED-5066',
 'MED-5305',
 'MED-2310',
 'MED-884',
 'MED-3153',
 'MED-5238',
 'MED-4098',
 'MED-4871',
 'MED-847',
 'MED-5230',
 'MED-5028',
 'MED-860',
 'MED-4990',
 'MED-1292',
 'MED-5287',
 'MED-5359',
 'MED-821',
 'MED-708',
 'MED-4496',
 'MED-4922',
 'MED-975',
 'MED-5126',
 'MED-949',
 'MED-5205',
 'MED-4262',
 'MED-4892',
 'MED-5225',
 'MED-4862',
 'MED-945',
 'MED-904',
 'MED-835',
 'MED-754',
 'MED-4928',
 'MED-4866',
 'MED-3642',
 'MED-5356',
 'MED-4697',
 'MED-1299',
 'MED-869',
 'MED-4097',
 'MED-948',
 'MED-4714',
 'MED-995',
 'MED-5226',
 'MED-5125',
 'MED-757',
 'MED-5244',
 'MED-5354',
 'MED-5283',
 'MED-707',
 'MED-5299',
 'MED-897',
 'MED-818',
 'MED-1298',
 'MED-872',
 'MED-976',
 'MED-4580',
 'MED-5199',
 'MED-5286',
 'MED-825',
 'MED-4079',
 'MED-4448',
 'MED-5207',
 'MED-970',
 'MED-1009',
 'MED-5200',
 'MED-823',
 'MED-950',
 'MED-4699',
 'MED-913',
 'MED-5209',
 'MED-4414',
 'MED-827',
 'MED-1300',
 'MED-5171',
 'MED-5361',
 'MED-761',
 'MED-865',
 '

319

In [31]:
import ast

# Get grad norm dict

# pth = "/gallery_louvre/jinyoung.kim/ood/eval_retrieval/BGE_M3/"
pth = "/gallery_louvre/dayoon.ko/research/sds/eval_retrieval/gradnorm/"
n_aug = 16

def load_file(pth):
    with open(pth) as f:
        lines = f.readlines() 
    return lines 
    
def get_gradnorms(pth):
    lines = load_file(pth)
    gradnorms = []
    for l in lines:
        try: 
            l = l.replace('\'', '"')
            item = json.loads(l)
            gradnorms.append(float(item["grad_norm"]))
        except:
            continue
    ids = []
    for l in lines:
        try:
            sid = l.index("doc_id: ") + len("doc_id: ") 
            eid = sid
            ids.append(l[sid:].strip())
        except:
            continue

    gradnorms_selected = []
    ids_selected = []
    func = lambda x: sum(x) / len(x)
    for i in range(0, len(ids), n_aug):
        gradnorms_selected.append(func(gradnorms[i:i+n_aug]))
        ids_selected.append(ids[i])
    gradnorm_dict = {cid: gradnorm for cid, gradnorm in zip(ids_selected, gradnorms_selected)}
    return gradnorm_dict
    
gradnorm_dict = get_gradnorms(pth + txt_name)

In [34]:
set(wrong_doc_ids) & set(gradnorm_dict.keys())

set()

In [60]:
len(sorted_wrong_doc_ids)

505

In [67]:
# Load queries that don't match failure documents
# 12334
jsonl_path = f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/corpus.jsonl"
with open(jsonl_path) as f:
    js = [json.loads(i) for i in f.readlines()]
print(len(js))

tsv_path = f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/qrels/test.tsv"
df = pd.read_csv(tsv_path, sep='\t')
print(len(df))
df = df[df["score"] >= 1]
print(len(df))
query_ids = df['query-id'].tolist()
corpus_ids = df['corpus-id'].tolist()
print(len(set(corpus_ids)))
scores = df['score'].tolist()

3633
12334
12334
3128
